In [1]:
import string
import re

In [2]:
def vectors_db(embed_path, db_path):
    query_instruction = "为这个句子生成表示以用于检索相关文章："
    embeddings = HuggingFaceBgeEmbeddings(model_name= embed_path,
                                            model_kwargs={'device': 'cuda'},
                                            query_instruction=query_instruction) 
    vectordb = Chroma(persist_directory=db_path, embedding_function=embeddings)
    return vectordb
import random
import json
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
import re
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
from sentence_transformers import CrossEncoder
import logging
import argparse
from FlagEmbedding import FlagModel
from sklearn.cluster import KMeans
import logging
EMBED_PATH = "/data/yangdi/bge-large-zh-v1.5"
ALL_DB_PATH = "/data/yangdi/ALL-bge-1.5-300"
all_vectordb = vectors_db(EMBED_PATH, ALL_DB_PATH)
print('All knowledge database gets ready !')

with open("/data/yangdi/data/psyqa_test.json", encoding='utf-8') as f:
    datas = json.load(f)
# QWEN_PATH = "/data/yangdi/qwen/Qwen1___5-14B-Chat"
# query_model = AutoModelForCausalLM.from_pretrained(
#     QWEN_PATH,
#     torch_dtype="auto",
#     device_map="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(QWEN_PATH)
RERANKER_PATH = '/data/pretrained_models/maidalun/bce-reranker-base_v1'
model = CrossEncoder(RERANKER_PATH, max_length=512)

/home/yangdi/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


All knowledge database gets ready !


In [3]:

def reranker_top2(sentence_pairs):
# calculate scores of sentence pairs
    sort_scores = {}
    scores = model.predict(sentence_pairs)
    for i, score in enumerate(scores):
        sort_scores[i] = score
    scores_ = sorted(sort_scores.items(), key=lambda item: item[1], reverse=True)
    index1, index2 = scores_[0][0], scores_[1][0]
    return index1, index2

from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from dashscope import Generation
from http import HTTPStatus
from dashscope import Generation
from dashscope.api_entities.dashscope_response import Role
import dashscope
dashscope.api_key="sk-9f54f947734647c89c7d1e37a8054c41"
def query_Qwen_7B(prompt, stop_=None):
    # Instead of using model.chat(), we directly use model.generate()
    # But you need to use tokenizer.apply_chat_template() to format your inputs as shown below
    device = "cuda" # the device to load the model onto

    # # Now you do not need to add "trust_remote_code=True"
    # query_model = AutoModelForCausalLM.from_pretrained(
    #     QWEN_PATH,
    #     torch_dtype="auto",
    #     device_map="auto"
    # )
    # tokenizer = AutoTokenizer.from_pretrained(QWEN_PATH)
    messages = [
        {"role": "system", "content": "You are a psychological expert and specialized in psychological knowledge."},
        {"role": "user", "content": prompt}
    ]

    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    response = Generation.call(
        'qwen1.5-14b-chat',
        messages=messages,
        stop=stop_
    )
    if response.status_code == HTTPStatus.OK:
        response = response.output["text"]
        print(response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    return response


In [ ]:
def get_answer(question):
    prompt = f"""
    根据下面知识回答问题：
    问题：{question}
    知识：{knowledge}
    """
    query = model_type('Qwen')
    res = query(prompt)
    return res